# Exercise 6

In this exercise, we will put in practice a popular clustering technique: k-means.

Remember that k-means is a two steps process. After initialization:  
1) assign each point to its closest cluster  
2) move each cluster to the average position of the points assigned to it  

Note that K-means requires you to specify the desired number of clusters k, which should be, in practice, determined from an apriori knowledge about the data or via cross-validation (not covered in this homework).

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn import preprocessing
from sklearn.datasets import make_blobs

np.random.seed(42)

%matplotlib inline

### TODO

Complete the two methods `assign_points` and `compute_centroids` below. Specifically:  

*  `assign_points(X, centroids)` assigns each data points in X to its closest centroid (in terms of Euclidean distance). The output is a list of indices, where each index is in the range [0,...,K-1], K being the number of clusters.
*  `compute_centroids(X, labels, num_clusters)` re-compute the position of each centroid. The new position of a centroid is computed as the mean of the data points that are assigned to it (recall that assignement is represented by the `labels` array).


In [6]:
"""
Generate random 2D data points
"""
def gen_data(num_samples,num_blobs=3,random_state=42):
    X, y = make_blobs(n_samples=num_samples, random_state=random_state, centers=num_blobs, cluster_std=5.0)
    return X,y

"""
Compute euclidean distance between a point and a centroid.
point,centroid: 1D Numpy array containing coordinates [x,y]
"""
def euclidean_distance(point, centroid):
    return np.sqrt(np.sum((point - centroid)**2))

def init_centroids(X,num_clusters):
    rand_indices = np.random.choice(X.shape[0], num_clusters,replace=False)
    return X[rand_indices,:]

"""
Assign objects to their closest cluster center according to the Euclidean distance function.
"""
def assign_points(X, centroids):
    labels = []
    for point in range(0, X.shape[0]):
        # Hint: you can use the function euclidean_distance(...) 
        #       to compute the distance between a point and 
        #       a centroid.
        
        # YOUR CODE HERE
        dists = [euclidean_distance(X[point], centroid) for centroid in centroids]
        labels.append(np.argmin(dists))
    
    return np.array(labels)

"""
Update the position of a centroid according to the average position of the points
of that cluster.
"""
def compute_centroids(X, labels, num_clusters):
    centroids = np.zeros((num_clusters,2))
    for i in range(num_clusters):
        # YOUR CODE HERE
        new_point_idx = np.where(labels == i)[0]
        new_points = X[new_point_idx]
        centroids[i] = np.mean(new_points, axis=0)
    
    return centroids

"""
Runs one instance of k-means
X:            input data of shape Mx2, M the number of examples
num_clusters: number of clusters to compute
"""
def k_means(X, num_clusters):

    # Initialize centroids to randomly chosen data points
    centroids  = init_centroids(X,num_clusters)

    # Bookkeeping
    num_iter  = 0
    positions = [centroids]
    
    while True:
        
        labels = assign_points(X, centroids)
        
        new_centroids = compute_centroids(X, labels, num_clusters)

        num_iter += 1
        
        # Termination criterion
        if np.all(centroids == new_centroids):
            break
            
        centroids = new_centroids
        positions.append(centroids)
        
    return [labels, centroids, num_iter, positions]

In [7]:
num_samples = 150
num_blobs   = 3
X,y = gen_data(num_samples,num_blobs=num_blobs)

total_iterations = 10
num_clusters     = 3
[cluster_label, new_centroids, num_iter, positions] = k_means(X,num_clusters)
print("Convergence in %i iteration(s)" % num_iter)

Convergence in 10 iteration(s)


### Plotting

With the following code you can plot the result of the clustering procedure.  

Cluster centroids are displayed as red circles. Their positions across the various steps of the optimization are displayed as red lines. Data points are colored according to their cluster assignement.

In [10]:
_ = plt.scatter(X[:, 0], X[:, 1], c=cluster_label)
_ = plt.scatter(new_centroids[:,0], new_centroids[:,1], marker='o', s=150, c='r', edgecolors='w', linewidths=2)

plt.plot(np.array(positions)[:,0,0],np.array(positions)[:,0,1],'r',linewidth=3)
plt.plot(np.array(positions)[:,1,0],np.array(positions)[:,1,1],'r',linewidth=3)
plt.plot(np.array(positions)[:,2,0],np.array(positions)[:,2,1],'r',linewidth=3)

## Pen and Paper

Consider a set P of points as shown below:  
<img src="ex2.png" width="400">

#### Question 2.1

Recall that the DBSCAN algorithm classifies each data point into one of 3 types: core ,  border , and  noise . The classification is based on two parameters, a distance  r and a threshold  t . Set  r = 1 and  t = 3. What are the types of a, b, c in the figure, respectively?  

#### Question 2.2
Set  r  = 1 and   t  = 3, and consider the point set  P  in Question 2.  
1) Are points  d and  e  connected? If so, show that there is a point  p that can
reach both  d   and  e.  
2) How about points  f  and  g ?  

#### Question 2.3  
Set  r  = 1 and  t  = 3. Show the output of DBSCAN on the point set  P in Question 2 (i.e., what are the points in each of the clusters returned?).